# Data Preprocessing Tools

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Part 1 - Data Preprocessing

### Importing dataset

data를 정제하기 쉬운 방법을 택하면 됨, 보통 컬럼 이름이 있는 데이터가 많다보니 pandas를 이용하는 편이 좋음

In [2]:
dataset = pd.read_csv('../Data/Part 1 - Artificial Neural Networks (ANN)/Churn_Modelling.csv')
df = pd.DataFrame(dataset)
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


머신러닝에 필요하지 않은 데이터는 제거, 처음 3개의 컬럼은 필요하지 않을것으로 보임

In [3]:
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values
print(X,y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]] [1 0 1 ... 1 1 0]


---

### Encoding categorical data

##### 성별 컬럼 encoding 하기
머신러닝은 숫자 데이터만 처리할 수 있으므로, gender를 0,1 로 바꾸어 줌 \
0 / 1 은 구분을 위한 숫자일 뿐 `크기`를 갖고 있지 않다는 점을 확인

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [5]:
print(X)  # gender 순서는 알파벳 순서를 따름. 다르게 mapping 가능

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


##### One Hot Encoding -- 'Geography column'

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [7]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


##### Training / Test set으로 분리

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

##### Feature Scaling

###### Machine Learning 알고리즘은 `데이터의 크기`에 민감하기 때문
- age: 20 ~ 70
- salery: 10,000 ~ 100,1000 (큰 값)
###### 스케일 차이가 큰 값을 학습하면 모델이 큰 값에 더 많은 가중치를 부여하기 때문 == 여러가지 방법이 있으나, 나중에 학습

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---

## Part 2 - Building ANN
#### Initializing the ANN

In [10]:
ann = tf.keras.models.Sequential()

2024-11-25 15:02:00.113308: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-11-25 15:02:00.113336: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-11-25 15:02:00.113343: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-11-25 15:02:00.113399: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-25 15:02:00.113565: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


#### Adding input layer and first hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

#### Adding second hidden layer

In [12]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

#### Adding OUTPUT layer

In [13]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

---
## Part 3 - Training ANN

#### Compiling ANN

In [14]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#### Training ANN

In [15]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
  1/250 [..............................] - ETA: 1:36 - loss: 0.9437 - accuracy: 0.6250

2024-11-25 15:02:00.537115: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 1s 3ms/step - loss: 0.6629 - accuracy: 0.6556
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4506 - accuracy: 0.8087
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4329 - accuracy: 0.8089
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4316 - accuracy: 0.8074
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4321 - accuracy: 0.8081
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4323 - accuracy: 0.8059
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4332 - accuracy: 0.8065
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4328 - accuracy: 0.8075
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4328 - accuracy: 0.8061
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4325 - accuracy: 0.8080
E

---
Part 4 - Making predictions and evaluating Model

**Predict** \
Geography: France, Credit Score: 600, Gender: Male, Age: 40, Tenure: 3 years, Balance: $60,000, Number of Products: 2 \
Credit card: Yes, Estimated Salary: $50,000

In [16]:
print(ann.predict(sc.transform( [[1,0,0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]] )) > 0.5)

1/1 [==============================] - 0s 84ms/step
[[False]]


2024-11-25 15:03:20.820982: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


**Important**
###### 1. 훈련에 2D array가 사용되었으므로, 예측에도 2D array 필요
###### 2. france 를 one hot encoding 한 숫자로 표현([1,0,0])

#### Predicting test set results

In [17]:
y_pred = ann.predict(X_test)
y_pred = (y_pred>0.5)
print(np.concatenate( (y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)) ,1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [1 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


#### Making Confusion Matrix

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1417  178]
 [ 265  140]]


0.7785